In [27]:
from bs4 import BeautifulSoup
from selenium import webdriver ## 브라우저 제어
from selenium.webdriver.chrome.service import Service as ChromeService ## 크롬 브라우저 제어
from selenium.webdriver.common.keys import Keys ## 키보드 제어
from selenium.webdriver.support import expected_conditions as EC ## 웹페이지 조건 기다림
from selenium.webdriver.support.ui import WebDriverWait ## 웹페이지 대기
from selenium.webdriver.common.by import By ## 웹페이지 요소

from webdriver_manager.chrome import ChromeDriverManager


import warnings
warnings.filterwarnings(action="ignore")
import re
import pandas as pd
import datetime
import time
import os
from tqdm import tqdm

today = datetime.date.today().strftime("%Y-%m-%d")

options = webdriver.ChromeOptions()
options.add_argument("--disable-web-security") ## 웹 보안 해제
options.add_argument("--disable-site-isolation-trials") ## 사이트 격리 해제
options.add_argument("--window-size=1440,1200") ## 창 크기

# options.add_argument('headless') ## 헤드리스

Keys = Keys()
options.add_argument(
    "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2564.116 Safari/537.37"
) ## 가짜 User-Agent 실행 (크롬 49 버전으로 위장)


driver = webdriver.Chrome(
    service=ChromeService(ChromeDriverManager().install()), options=options
) ## 크롬드라이버를 설치하고 옵션을 적용하여 실행

driver.implicitly_wait(10) ## 웹페이지가 로드되는 시간을 암묵적 대기, 5초 기다림

In [28]:
BASE_URL = '''https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D&tab=position'''

driver.get(BASE_URL)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [48]:
time.sleep(1)

## 무한 스크롤 아래로 내리기
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

KeyboardInterrupt: 

In [31]:
## 페이지 소스 가져오기
htmls = driver.page_source
soups = BeautifulSoup(htmls, 'html.parser')

items = soups.find_all('div', {'class': 'JobCard_container__FqChn JobCard_container--variant-card__znjV9'})

In [32]:
len(items)

130

In [33]:
items = list({item.text: item for item in items}.values())
len(items)

121

In [34]:
target_url_list = []
for item in items:
    url = 'https://www.wanted.co.kr'+item.find('a')['href']
    target_url_list.append(url)

In [22]:
target_url_list

[]

In [36]:
driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a').text

'부릉'

In [37]:
driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/h1').text

'Data Scientist'

In [38]:
company_name = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a').text
title = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/header/h1').text

In [39]:
driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()

In [55]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

## 상세 더보기 버튼 누르기
try:
    driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()
except:
    pass

position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
                    .find('article', {'class': 'JobDescription_JobDescription__VWfcb'})

In [56]:
position_detail.text

'포지션 상세인터엑스는 제조AI 및 자율공장 전문 파트너 입니다.누구나 쉽게 빅데이터를 분석하고 쉽게 현장에 적용할 수 있도록 인공지능(AI) 기반 제조 빅데이터 분석 서비스 플랫폼을 제공하고 있습니다.인터엑스의 제조 빅데이터 분석 서비스는 현장의 데이터 수집부터 데이터 품질관리, Advanced 빅데이터 분석,  AI Library를 바탕으로 한 지능화된 공정 예측과 최적화를 통해 기업의 성장을 지원합니다.저희의 목표는 IoT(사물 인터넷)의 결합으로 개선된 프로세스를 현장에 쉽게 적용하여 스마트 제조 환경을 구현하는 것과 제조 빅데이터가 가지는 무궁무진한 가능성을 제조 기업 모두가 누리며 함께 성장하는 성공 스토리를 만드는데 있습니다.저희와 함께 제조AI 성공 스토리를 만들어갈 유능한 분들의 많은 지원 부탁 드립니다.주요업무[함께하시면 맡게 될 업무입니다!]- 이미지 및 영상 데이터 처리- 제조업 AI, 머신러닝, 딥러닝 알고리즘 최신 기술 연구 및 개발- 제조업 Defect classification, detection, segmentation, measurement 에 적용 가능한 알고리즘 연구 및 개발- 알고리즘 고도화자격요건[이런 분이 필요해요!] - 이미지 데이터에 대한 기초 지식이 있으신 분- Python을 활용하여 머신러닝/딥러닝 프로젝트 경험이 있으신 분- OpenCV, Numpy, Tensorflow 및 Pytorch 사용 경험이 있으신 분- CNN 및 transformer 사용 경험이 있으신 분- 최신 기술에 관심이 많은 분- 주도적으로 일을 만들고, 책임감 있게 리드하시는 분- 다른 직군과 함께 문제를 정의하고 해결책을 찾을 수 있는 분 [업무에 꼭 필요한 기술 스택 입니다]- Python- OpenCV, Numpy- Pytorch or Tensorflow상세 정보 더 보기'

In [ ]:
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

# ## 상세정보 클릭
# driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()

# ## 페이지 소스 업데이트
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
# position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
#                     .find('article', {'class': 'JobDescription_JobDescription__VWfcb'})

In [58]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

## 상세정보 클릭
driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()

## 페이지 소스 업데이트
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
                    .find('article', {'class': 'JobDescription_JobDescription__VWfcb'})

In [59]:
team_info = position_detail.find('p').text

In [60]:
team_info = position_detail.find('p').text
main_job_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[0].find('h3').text
main_job_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[0].find('p').text
requirement_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[1].find('h3').text
requirement_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[1].find('p').text

preferred_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[2].find('h3').text
preferred_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[2].find('p').text
wellbeing_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[3].find('h3').text
wellbeing_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[3].find('p').text

In [62]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# JobSkillTags_JobSkillTags__V1tZr
skill_stacks = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
                    .find('article', {'class': 'JobSkillTags_JobSkillTags__V1tZr'})

skill_title = skill_stacks.find('h2').text
skill_list = [i.text for i in skill_stacks.find('ul').find_all('li')]

In [64]:
def detail_crawl(driver, index):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

## 상세정보 클릭
    driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()
    time.sleep(1)

## 페이지 소스 업데이트
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
        .find('article', {'class': 'JobDescription_JobDescription__VWfcb'})
    team_info = position_detail.find('p').text
    team_info = position_detail.find('p').text
    main_job_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[index].find('h3').text
    main_job_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[index].find('p').text
    requirement_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[index].find('h3').text
    requirement_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[index].find('p').text

    preferred_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[index].find('h3').text
    preferred_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[index].find('p').text
    wellbeing_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[index].find('h3').text
    wellbeing_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[index].find('p').text

In [65]:
def detail_crawl(driver, idx):

    team_info = None
    main_job_title = None
    main_job_content = None
    requirement_title = None
    requirement_content = None
    preferred_title = None
    preferred_content = None
    wellbeing_title = None
    wellbeing_content = None
    skill_title = None
    skill_list = None
    address = None
    current_url = None


    ## 상세 더보기 버튼 누르기
    try:
        driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button/span[2]').click()
    except Exception as e:
        print(idx, e)
        pass

    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    company_name = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a').text
    title = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/header/h1').text
    try:
        target_elements1 = driver.find_element('xpath', '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[1]/h3')
        # 요소가 화면에 보이도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView();", target_elements1)

        # 요소가 보일때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,\
        '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[1]/h3')))

        ## 스크린샷 찍기
        # driver.save_screenshot(f'screenshot/{idx}_{company_name}.png')

        ## 저장할때까지 기다리기
        time.sleep(1)
    except Exception as e:
        print("스크린샷 에러:", idx, e)

    try:
        position_detail = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
                            .find('article', {'class': 'JobDescription_JobDescription__VWfcb'})
        team_info = position_detail.find('p').text
        main_job_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[0].find('h3').text
        main_job_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[0].find('p').text
        requirement_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[1].find('h3').text
        requirement_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[1].find('p').text
        
        preferred_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[2].find('h3').text
        preferred_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[2].find('p').text
        wellbeing_title = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[3].find('h3').text
        wellbeing_content = position_detail.find_all('div', {'class': 'JobDescription_JobDescription__paragraph__Iwfqn'})[3].find('p').text
        
        ## 기술스택
        skill_stacks = soup.find('section', {'class': 'JobContent_descriptionWrapper__SM4UD'})\
                            .find('article', {'class': 'JobSkillTags_JobSkillTags__V1tZr'})
        
        if skill_stacks:

            skill_title = skill_stacks.find('h2').text
            skill_list = [i.text for i in skill_stacks.find('ul').find_all('li')]

        address = soup.find('article', {'class': 'JobWorkPlace_JobWorkPlace__5fs5x'}).find('span').text
        current_url = driver.current_url
    except Exception as e:
        print("상세정보 에러:", idx, e)
        pass

    return company_name, title, team_info, main_job_title, main_job_content, requirement_title, requirement_content, preferred_title, preferred_content, wellbeing_title, wellbeing_content, skill_title, skill_list, address, current_url

In [66]:
target_url_list

['https://www.wanted.co.kr/wd/224337',
 'https://www.wanted.co.kr/wd/219966',
 'https://www.wanted.co.kr/wd/215835',
 'https://www.wanted.co.kr/wd/218916',
 'https://www.wanted.co.kr/wd/224841',
 'https://www.wanted.co.kr/wd/223481',
 'https://www.wanted.co.kr/wd/222883',
 'https://www.wanted.co.kr/wd/223175',
 'https://www.wanted.co.kr/wd/161548',
 'https://www.wanted.co.kr/wd/191674',
 'https://www.wanted.co.kr/wd/216979',
 'https://www.wanted.co.kr/wd/201396',
 'https://www.wanted.co.kr/wd/215210',
 'https://www.wanted.co.kr/wd/193901',
 'https://www.wanted.co.kr/wd/142793',
 'https://www.wanted.co.kr/wd/191938',
 'https://www.wanted.co.kr/wd/219963',
 'https://www.wanted.co.kr/wd/85810',
 'https://www.wanted.co.kr/wd/224170',
 'https://www.wanted.co.kr/wd/180914',
 'https://www.wanted.co.kr/wd/208247',
 'https://www.wanted.co.kr/wd/215111',
 'https://www.wanted.co.kr/wd/221269',
 'https://www.wanted.co.kr/wd/210507',
 'https://www.wanted.co.kr/wd/212760',
 'https://www.wanted.co.kr

In [67]:
data_list = []

for idx, target in enumerate(tqdm(target_url_list)):
    try :
        driver.get(target)
        company_name, title, team_info, main_job_title, main_job_content, requirement_title, requirement_content, preferred_title, preferred_content, wellbeing_title, wellbeing_content, skill_title, skill_list, address, current_url = detail_crawl(driver, idx)
        time.sleep(1)
        data = {
            'company_name': company_name,
            'title': title,
            'team_info': team_info,
            'Main Job Title': main_job_title,
            'Main Job Content': main_job_content,
            'Requirement Title': requirement_title,
            'Requirement Content': requirement_content,
            'Preferred Title': preferred_title,
            'Preferred Content': preferred_content,
            'Wellbeing Title': wellbeing_title,
            'Wellbeing Content': wellbeing_content,
            'Skill Title': skill_title,
            'Skill List': skill_list,
            'Address': address,
            'Current URL': current_url
        }
        data_list.append(data)
    except Exception as e:
        print(company_name, e)
        pass    

res_df = pd.DataFrame(data_list)

  0%|          | 0/121 [00:00<?, ?it/s]

0 Message: element click intercepted: Element <span class="Button_Button__interaction__kkYaa"></span> is not clickable at point (427, 1030). Other element would receive the click: <div class="ApplyBtn_container__pWc4W">...</div>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x007DB8E3+45827]
	(No symbol) [0x0076DCC4]
	(No symbol) [0x0066150F]
	(No symbol) [0x006A8052]
	(No symbol) [0x006A63D8]
	(No symbol) [0x006A425B]
	(No symbol) [0x006A3823]
	(No symbol) [0x006985EF]
	(No symbol) [0x006C2DFC]
	(No symbol) [0x00698075]
	(No symbol) [0x006C3094]
	(No symbol) [0x006DC034]
	(No symbol) [0x006C2B96]
	(No symbol) [0x00696998]
	(No symbol) [0x0069751D]
	GetHandleVerifier [0x00A94513+2899763]
	GetHandleVerifier [0x00AE793D+3240797]
	GetHandleVerifier [0x008613B4+593364]
	GetHandleVerifier [0x008682DC+621820]
	(No symbol) [0x007770A4]
	(No symbol) [0x007737A8]
	(No symbol) [0x00773947]
	(No symbol) [0x007659FE]
	BaseThreadInitThunk [0x7725FCC9+25]
	RtlGetAppContaine

  1%|          | 1/121 [00:06<12:46,  6.39s/it]

1 Message: element click intercepted: Element <span class="Button_Button__interaction__kkYaa"></span> is not clickable at point (427, 1030). Other element would receive the click: <div class="ApplyBtn_container__pWc4W">...</div>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x007DB8E3+45827]
	(No symbol) [0x0076DCC4]
	(No symbol) [0x0066150F]
	(No symbol) [0x006A8052]
	(No symbol) [0x006A63D8]
	(No symbol) [0x006A425B]
	(No symbol) [0x006A3823]
	(No symbol) [0x006985EF]
	(No symbol) [0x006C2DFC]
	(No symbol) [0x00698075]
	(No symbol) [0x006C3094]
	(No symbol) [0x006DC034]
	(No symbol) [0x006C2B96]
	(No symbol) [0x00696998]
	(No symbol) [0x0069751D]
	GetHandleVerifier [0x00A94513+2899763]
	GetHandleVerifier [0x00AE793D+3240797]
	GetHandleVerifier [0x008613B4+593364]
	GetHandleVerifier [0x008682DC+621820]
	(No symbol) [0x007770A4]
	(No symbol) [0x007737A8]
	(No symbol) [0x00773947]
	(No symbol) [0x007659FE]
	BaseThreadInitThunk [0x7725FCC9+25]
	RtlGetAppContaine

  2%|▏         | 2/121 [00:11<11:34,  5.83s/it]

...

12 Message: element click intercepted: Element <span class="Button_Button__interaction__kkYaa"></span> is not clickable at point (427, 1030). Other element would receive the click: <div class="ApplyBtn_container__pWc4W">...</div>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x007DB8E3+45827]
	(No symbol) [0x0076DCC4]
	(No symbol) [0x0066150F]
	(No symbol) [0x006A8052]
	(No symbol) [0x006A63D8]
	(No symbol) [0x006A425B]
	(No symbol) [0x006A3823]
	(No symbol) [0x006985EF]
	(No symbol) [0x006C2DFC]
	(No symbol) [0x00698075]
	(No symbol) [0x006C3094]
	(No symbol) [0x006DC034]
	(No symbol) [0x006C2B96]
	(No symbol) [0x00696998]
	(No symbol) [0x0069751D]
	GetHandleVerifier [0x00A94513+2899763]
	GetHandleVerifier [0x00AE793D+3240797]
	GetHandleVerifier [0x008613B4+593364]
	GetHandleVerifier [0x008682DC+621820]
	(No symbol) [0x007770A4]
	(No symbol) [0x007737A8]
	(No symbol) [0x00773947]
	(No symbol) [0x007659FE]
	BaseThreadInitThunk [0x7725FCC9+25]
	RtlGetAppContain

 11%|█         | 13/121 [01:12<09:59,  5.55s/it]

13 Message: element click intercepted: Element <span class="Button_Button__interaction__kkYaa"></span> is not clickable at point (427, 1030). Other element would receive the click: <div class="ApplyBtn_container__pWc4W">...</div>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x007DB8E3+45827]
	(No symbol) [0x0076DCC4]
	(No symbol) [0x0066150F]
	(No symbol) [0x006A8052]
	(No symbol) [0x006A63D8]
	(No symbol) [0x006A425B]
	(No symbol) [0x006A3823]
	(No symbol) [0x006985EF]
	(No symbol) [0x006C2DFC]
	(No symbol) [0x00698075]
	(No symbol) [0x006C3094]
	(No symbol) [0x006DC034]
	(No symbol) [0x006C2B96]
	(No symbol) [0x00696998]
	(No symbol) [0x0069751D]
	GetHandleVerifier [0x00A94513+2899763]
	GetHandleVerifier [0x00AE793D+3240797]
	GetHandleVerifier [0x008613B4+593364]
	GetHandleVerifier [0x008682DC+621820]
	(No symbol) [0x007770A4]
	(No symbol) [0x007737A8]
	(No symbol) [0x00773947]
	(No symbol) [0x007659FE]
	BaseThreadInitThunk [0x7725FCC9+25]
	RtlGetAppContain

 12%|█▏        | 14/121 [01:19<10:10,  5.71s/it]


KeyboardInterrupt: 